#### Functions

In [ ]:
from adal import AuthenticationContext
from msal import PublicClientApplication
import requests
import json
import pandas as pd
import hvplot.pandas
import holoviews as hv
import panel as pn
from tqdm.notebook import tqdm

pn.extension()

def get_timeseries_token(tenant_id, client_id, client_secret):
    auth_context = AuthenticationContext("https://login.microsoftonline.com/"+tenant_id)
    resource="https://api.timeseries.azure.com/"
    access_token_json=auth_context.acquire_token_with_client_credentials(resource, client_id, client_secret)
    token='Bearer '+access_token_json['accessToken']
    return token

def get_time_series_properties(token, search_from="2007-06-21T00:00:00.000Z", search_to="2009-06-27T00:00:00.000Z"):
    body={"searchSpan": {"from": search_from,"to": search_to}}
    url='https://53c844e6-9dc6-48de-aa60-1582494b83ae.env.timeseries.azure.com/eventSchema?api-version=2020-07-31'
    headers = {"Authorization": token}
    response = requests.post(url, headers=headers, json=body)
    response_json=json.loads(response.text)
    return response_json["properties"]

def generate_query_body(search_from="2007-06-21T00:00:00.000Z", search_to="2009-06-27T00:00:00.000Z", timeseries_id='Rig01', 
                        properties=[{"name": "ROP","type": "Double"},{"name": "WOB","type": "Double"},
                                            {"name": "BIT_DEPTH", "type": "Double"},{"name": "FLOW","type": "Double"},
                                            {"name": "HOLE_DEPTH","type": "Double"},{"name": "RPM","type": "Double"},
                                            {"name": "SPP","type": "Double"},{"name": "TRQ","type": "Double"}]):
    body_template={"getEvents": {"timeSeriesId": [timeseries_id],"searchSpan": {"from": search_from,"to": search_to},
                    "projectedProperties": properties}}
    return body_template

def time_series_execute(body, token):
    url='https://53c844e6-9dc6-48de-aa60-1582494b83ae.env.timeseries.azure.com/timeseries/query?api-version=2020-07-31'
    headers = {"Authorization": token}
    response = requests.post(url, headers=headers, json=body)
    response_json=json.loads(response.text)
    return response_json

def json_response_to_df(response_json):
    _dict={}
    for series in response_json['properties']:
        _dict[series['name']]=series['values']
    _dict['timestamp']=response_json['timestamps']
    df_response=pd.DataFrame.from_dict(_dict)
    df_response['timestamp']=pd.to_datetime(df_response['timestamp'])
    return df_response.set_index('timestamp')

def get_timeseries_as_df(tenant_id, client_id, client_secret, 
                         search_from="2007-06-24T00:00:00.000Z", search_to="2009-06-27T00:00:00.000Z", timeseries_id='Rig01'):
    token = get_timeseries_token(tenant_id, client_id, client_secret)
    properties=get_time_series_properties(token, search_from=search_from, search_to=search_to)
    body=generate_query_body(search_from=search_from, search_to=search_to, timeseries_id=timeseries_id, properties=properties)
    response_json=time_series_execute(body,token)
    df_response=json_response_to_df(response_json)
    df_response=df_response[['ROP','RPM','WOB','TRQ','FLOW','SPP','BIT_DEPTH','HOLE_DEPTH']]
    return df_response

### Set up panels

In [ ]:
tenant_id="<Enter Tenant-ID>"
client_id="<Enter Client-ID>"
client_secret="<Enter Client Secret>"

In [ ]:
df=pd.read_csv('field_rig_wellbore_section_time.csv', delimiter=';')

field_select = pn.widgets.MultiSelect(name='Field', value=['VOLVE'],
                                      options=[item for item in df['Field'].unique()], size=8)
rig_select = pn.widgets.MultiSelect(name='Rig', 
                                    options=[item for item in df[df.Field.isin(field_select.value)]['Rig'].unique()], size=8)
wellbore_select = pn.widgets.MultiSelect(name='Wellbore', 
                                         options=[item for item in df[df.Rig.isin(rig_select.value)]['Wellbore'].unique()]
                                         , size=8)
section_select = pn.widgets.MultiSelect(name='Section', 
                                         options=[item for item in df[df.Wellbore.isin(wellbore_select.value)]['Section'].unique()]
                                         , size=8)
date_picker_from = pn.widgets.DatePicker(name='From Date', width=140)
date_picker_to = pn.widgets.DatePicker(name='To Date', width=140)
df_widget = pn.widgets.DataFrame(df.head(0), name='DataFrame', height=300, width=800)

df_data_selection=get_timeseries_as_df(tenant_id, client_id, client_secret)
plot=df_data_selection.hvplot.line()
plot_panel=pn.panel(plot)

plot_button = pn.widgets.Button(name='Plot', button_type='primary')

def field_callback(field_event):
    rig_select.options = [item for item in df[df.Field.isin(field_event.new)]['Rig'].unique()]

field_select.param.watch(field_callback, 'value')

def rig_callback(rig_event):
    wellbore_select.options = [item for item in df[(df.Rig.isin(rig_event.new))&(df.Field.isin(field_select.value))]['Wellbore'].unique()]
    
rig_select.param.watch(rig_callback, 'value')

def wellbore_callback(wellbore_event):
    df_widget.value = df[df.Wellbore.isin(wellbore_event.new)]
    section_select.options = [item for item in df[df.Wellbore.isin(wellbore_event.new)]['Section'].unique()]
    
wellbore_select.param.watch(wellbore_callback, 'value')

def section_callback(section_event):
    df_widget.value = df[(df.Section.isin(section_event.new))&(df.Wellbore.isin(wellbore_select.value))]
    
section_select.param.watch(section_callback, 'value')

def from_date_callback(from_date_event):
    df_widget.value = df[(df.Section.isin(section_select.value))&(df.Wellbore.isin(wellbore_select.value)&(df['Start time']>from_date_event.new))]
    
date_picker_from.param.watch(from_date_callback, 'value')

def to_date_callback(to_date_event):
    df_widget.value = df[(df.Section.isin(section_select.value))&(df.Wellbore.isin(wellbore_select.value)&(df['Start time']<to_date_event.new))]
    
date_picker_to.param.watch(to_date_callback, 'value')

def get_start_and_time(event):
    start_time=df_widget.value['Start time'].min().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    end_time=df_widget.value['End time'].max().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    return [start_time, end_time]

def get_df(event):
    time=get_start_and_time(event)
    df = get_timeseries_as_df(tenant_id, client_id, client_secret, search_from=time[0], search_to=time[1], timeseries_id='Rig01')
    return df

def get_plot(event):
    df = get_df(event)
    plot_panel.object=df.hvplot()

plot_button.param.watch(get_plot, 'clicks')

plot_panel.param.watch(plot_button, 'object')

panel =pn.Row(pn.Row(pn.Column(field_select, rig_select, wellbore_select, section_select, 
                         pn.Row(date_picker_from, date_picker_to)), 
               pn.Column(df_widget, plot_button, plot_panel)))

In [ ]:
panel.show()